# Anwendungsmodul: Dateien lesen und schreiben

In diesem Beispielnotebook präsentiere ich einige Beispiele, wie Textdateien und einfache Tabellen (CSV, TSV) mit Python gelesen und geschrieben werden können. In den früheren Notebooks sind schon Beispiele dazu vorgekommen, hier führe ich diese Methoden noch etwas aus.

Relevante Python-Dok-Links für weiteres Einlesen:
* [open()](https://docs.python.org/3/library/functions.html#open)
* [os und paths](https://docs.python.org/3/library/filesys.html)
* [csv](https://docs.python.org/3/library/csv.html)

## Teil 1: Lesen und Schreiben

Hier nochmal zur Erinnerung, wie eine Textdatei eingelesen werden kann:

In [ ]:
infile = open("moby_dick.txt", encoding="utf8")

for line in infile:
    print(line)

infile.close()

Sehen wir uns zuerst die `open()`-Funktion an. In der Dokumentation könnt ihr sehen, dass sie viele Optionale Argumente enthält, immer notwendig ist aber der Pfad zur Datei, meistens in Form eines Strings. Die meisten optionalen Argumente werdet ihr nie verwenden müssen, aber `encoding` sollte man sicherheitshalber explizit setzen. 

Eine kurze Erklärung zu Encodings: Computer lesen Zeichen nicht wie wir, sondern als Code. Je nach Encoding werden diese Zeichen anders repräsentiert. Nicht alle Encodings enthalten Codes für alle Zeichen. ASCII zum Beispiel enthält nur 95 Zeichen, die das lateinische Alphabet, einige Sonderzeichen und Zahlen umfassen. Windows verwendet standardmässig das hauseigene Windows-125x-System, während die meisten Plattformen heute das umfangreiche Unicode verwenden, das in Python unter anderem mit dem Namen "utf8" identifiziert wird. Unicode enthält auch Encodings für Zeichen von seltenen Alphabeten und Emojis.

Wir können in Python Strings auch in einem bestimmten Encoding codieren, damit sehen wir gleich den Unterschied:

In [ ]:
# Ohne spezielle Zeichen sehen wir keinen Unterschied
text = "Bald ist April."

print(text.encode("ascii"))

print(text.encode("latin-1"))

print(text.encode("utf8"))


In [ ]:
# Mit einem ä sieht es schon anders aus
# Ascii kann diesen String nicht mehr codieren!

text = "Der März ist bald vorbei."

# print(text.encode("ascii"))

print(text.encode("latin-1"))

print(text.encode("utf8"))

In [ ]:
# Fügen wir auch noch ein Emoji hinzu, scheitert auch Latin-1

text = "Der März ist bald vorbei. 😃"

# print(text.encode("ascii"))

# print(text.encode("latin-1"))

print(text.encode("utf8"))

Alle Encodings, die in Python standardmässig implementiert sind, findet ihr in der [Dokumentation](https://docs.python.org/3/library/codecs.html). Ich empfehle, das Encoding beim Lesen und Schreiben von Dateien immer explizit anzugeben, da Windows seinen eigenen Standard hat (statt Unicode, wie es bei UNIX-Systemen üblich ist) und sich Skripte daher unerwünscht anders verhalten können, je nach Gerät, auf dem man sie ausführt, wenn man kein Encoding explizit angibt.

Nochmal zurück zum Code zu Beginn. Wir sehen ausserdem, dass man Zeilen des Texts einfach iterieren kann, indem man eine For-Schleife über dem Datei-Objekt ausführt. Das Datei-Objekt verhält sich dabei übrigens wie ein Generator: Erst wenn die Zeile bearbeitet wird, wird sie in den Arbeitsspeicher geladen. Das ist effizient und nützlich, wenn man mit sehr langen Dateien arbeitet. Es kann aber auch Gründe geben, wieso man den Text nicht Zeile für Zeile, sondern als Fliesstext haben möchte. In dem Fall können wir folgende Methode verwenden:

In [ ]:
infile = open("moby_dick.txt", encoding="utf8")

text = infile.read()
text = text.replace("\n", " ")
print(text[:100])

infile.close()

Ich ersetze im obigen Code auch noch die Zeilenumbrüche (`\n`, `\t` für Tabs) durch Leerschläge, um den Text als einen durchgehenden String zu repräsentieren. Wenn einem Zeilen zwar doch wichtig sind, man aber ganz bestimmte möchte, kann man `readlines()` verwenden.

In [ ]:
infile = open("moby_dick.txt", encoding="utf8")

text = infile.readlines()
print(text[42:45])

infile.close()

Um Dateien zu schreiben, statt zu lesen, ist nur einem minimale Änderung notwendig: Wir müssen das optionale Argument `mode` auf `w` (für "write") setzen:

In [ ]:
outfile = open("test.txt", mode="w", encoding="utf8")

outfile.write("Erste Zeile!\n")
outfile.write("Zweite Zeile...\n")
outfile.write("Letzte Zeile :-(\n")

outfile.close()

Mit dem "write"-Modus wird die Datei zum Schreiben geöffnet, wobei sämtlicher bisheriger Inhalt gelöscht wird. Möchte man hingegen an einer Datei weiterschreiben, verwendet man den Modus `a` für "append".

Zum Abschluss dieses Kapitels noch eine alternative Schreibweise fürs Öffnen und Schliessen von Dateien, die sehr beliebt ist:

In [ ]:
with open("moby_dick.txt", encoding="utf8") as infile:
    for line in infile:
        print(line)

Diese Schreibweise erübrigt uns, die Datei nach dem Lesen wieder zu schliessen (und dadurch den Arbeitsspeicher freizugeben), indem die Datei einfach für alle Operationen innerhalb des Einschubs mit der geöffneten Datei ausgeführt werden, und sobald der Einschub verlassen wird, wird die Datei geschlossen. Versucht man dann noch, darauf zu lesen oder zu schreiben, erhält man eine Fehlermeldung

## Teil 2: Pfadoperationen

In manchen Fällen ist ein Programm komplizierter, als nur eine einzelne Datei zu öffnen und einzulesen. Die `os`-Bibliothek bietet eine Reihe von Operationen an, die es euch ermöglichen, mit Dateipfaden zu arbeiten. Das Modul `glob` hilft uns, relevante Dateien zu finden. (Alle Teil der Standard-Bibliothek)

In [ ]:
import glob
import os

# Mit glob können wir mehrere Dateien finden und Platzhalter verwenden
infiles = glob.glob("./Beispiele/*.txt")

print(infiles)

In [ ]:
# Überführen wir die Infos aus den Dateien in ein Dictionary
import pprint as pp

states = {}

for infile in infiles:
    
    # Wir benutzen os um den Namen der Datei zu erhalten
    name = os.path.basename(infile)
    name = name.replace(".txt", "")

    with open(infile, encoding="utf8") as inf:
        text = inf.read()

    states[name] = text

pp.pprint(states)

Die Möglichkeiten, die uns `os` bietet, sind zu zahlreich, um sie hier aufzuzählen. Man kann damit aber z.B. auch Ordner anlegen, oder überprüfen, ob gewisse Dateien oder Ordner vorhanden sind. Das `path`-Submodul bietet hauptsächlich Möglichkeiten, Strings zu manipulieren, wie oben im Beispiel, in dem der "Basename" der Datei extrahiert wird. Man kann mit `os.path.join()` aber zum Beispiel auch aus einer Sequenz einen Pfad bauen oder mit `os.path.split()` einen Dateipfad in seine Einzelteile zerlegen.

## Teil 3: Einfache Tabellen

Für die Arbeit mit Tabellen stellen die Tutor*innen euch ein spezifisches Notebook zur Verfügung, in dem ihr in den Umgang mit Pandas eingeführt werdet. Pandas ist ein sehr gutes externes Python-Modul, das die Arbeit mit tabellarischen Datensätzen vereinfacht. Möchte man aber nur eine einfache Tabelle einlesen, ist es etwas unumgänglich. Daher gehe ich hier noch in Kürze auf die Nutzung des `csv`-Moduls ein.

csv- und tsv-Tabellen sind Tabellen, die als Textdokumente gespeichert sind, und deren Spalten durch Kommas, resp. Tabulatoren getrennt sind. Wir können diese beiden mit `csv` einlesen. Eine Excel-Datei hingegen müssten wir erst csv-formatiert exportieren, um sie einlesen zu können. Hier empfiehlt sich dann meistens eher Pandas.

In eurem Materialordner findet ihr `cities.csv` zur Demonstration.

In [ ]:
import csv

with open("cities.csv", newline="", encoding="utf8") as infile:
    reader = csv.reader(infile)

    for row in reader:
        city = row[8]
        state = row[9]

        print(city, state)

Da wir hier eine csv-formatierte Datei haben, müssen wir keine optionalen Argumente setzen. Hat man hingegen eine tsv-formatierte Datei, oder - eher ungewöhnlich - eine Semikolon-separierte Datei, muss man das `delimiter`-Argument entsprechen setzen (`\t` für TSV z.B.).

Wie man sieht, ist hier aber das Ansteuern der einzelnen Felder ziemlich kompliziert, weil die Spalten nur über den Index definiert sind. `csv` bietet dazu auch noch den praktischen `DictReader`:

In [ ]:
import csv

with open("cities.csv", newline="", encoding="utf8") as infile:
    reader = csv.DictReader(infile)

    for row in reader:
        city = row["City"]
        state = row["State"]

        print(city, state)

Und genau so können wir auch Tabellen schreiben, zum Beispiel eine TSV-Datei:

In [ ]:
import csv

with open("letters.tsv", mode="w", newline="", encoding="utf8") as outfile:
    writer = csv.writer(outfile, delimiter="\t")

    writer.writerow(["A", "First Letter of the Alphabet."])
    writer.writerow(["Q", "Worst Letter of the Alphabet."])
    writer.writerow(["V", "Most Unnecessary Letter of the Alphabet."])

Wie aber schon erwähnt, lohnt es sich bei grösseren oder komplexeren Tabellen, diese mit dem `pandas`-Modul zu bearbeiten. (Siehe Vertiefungsmodul zur Arbeit mit Tabellen)